# Production d'un csv utilisable de la base FINESS

En l'état, l'export CSV de la [base FINESS][finess] n'est pas vraiment satisfaisant et utilisable.

- Le fichier n'est pas réellement un CSV.
    - Il est bizarrement découpé en deux sections qui correspondent au XML.
    - Les colonnes n'ont pas de nom.
- Le fichier est encodé au format windows.

[finess]: https://www.data.gouv.fr/en/datasets/finess-extraction-du-fichier-des-etablissements/

In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
dataset_api = "https://www.data.gouv.fr/api/1/datasets/finess-extraction-du-fichier-des-etablissements/"

In [ ]:
resources = (requests
    .get(dataset_api)
    .json()
    ['resources']
)

resource_geoloc = [ r for r in resources if r['type'] == 'main' and 'géolocalisés' in r['title']][0]

In [ ]:
headers = [
    'section',
    'nofinesset',
    'nofinessej',
    'rs',
    'rslongue',
    'complrs',
    'compldistrib',
    'numvoie',
    'typvoie',
    'voie',
    'compvoie',
    'lieuditbp',
    'commune',
    'departement',
    'libdepartement',
    'ligneacheminement',
    'telephone',
    'telecopie',
    'categetab',
    'libcategetab',
    'categagretab',
    'libcategagretab',
    'siret',
    'codeape',
    'codemft',
    'libmft',
    'codesph',
    'libsph',
    'dateouv',
    'dateautor',
    'maj',
    'numuai'
]

In [ ]:
geoloc_names = [
    'nofinesset',
    'coordxet',
    'coordyet',
    'sourcecoordet',
    'datemaj'
]

In [ ]:
raw_df = (pd
    .read_csv(resource_geoloc['url'],
              sep=";", encoding="Windows-1252", header=None, skiprows=1,
              dtype='str',
              names=headers)
    .drop(columns=['section'])
)

raw_df

In [ ]:
structures = (raw_df
    .iloc[:int(raw_df.index.size/2)]
)

structures

In [ ]:
geolocalisations = (raw_df
    .iloc[int(raw_df.index.size/2):]
    .drop(columns=raw_df.columns[5:])
    .rename(columns=lambda x: geoloc_names[list(raw_df.columns).index(x)])
)

geolocalisations

In [ ]:
clean_df = (structures
    .merge(geolocalisations, on="nofinesset", how="left")
)

clean_df

In [ ]:
clean_df.sample().T

In [ ]:
clean_df["siret"]

## Vérification de la qualité des données

In [ ]:
intersection = pd.Series(np.intersect1d(structures.nofinesset.values, geolocalisations.nofinesset.values))

intersection.shape

In [ ]:
only_structures = (structures
    [ ~structures.nofinesset.isin(intersection) ]
)

only_structures

In [ ]:
only_geolocalisations = (geolocalisations
    [ ~geolocalisations.nofinesset.isin(intersection) ]
)

only_geolocalisations

In [ ]:
geolocalisations_missing = []

## Export final

In [ ]:
clean_df.to_csv('finess-clean.csv', encoding='utf-8')